<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Google-Health/imaging-research/blob/master/ct-foundation/CT_Foundation_Demo.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/Google-Health/imaging-research/tree/master/ct-foundation"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## CT Foundation API Demo
The ipynb is a demonstration of using the
[CT Foundation API](https://github.com/Google-Health/imaging-research/tree/master/ct-foundation)
(this API computes embeddings for CT DICOMs).

The contents include how to:

-   Load the LIDC dataset from DICOMs stored in Google DICOM Store and labels stored in GCS
-   Generate embeddings for the image files
-   Train a small model using the embeddings

**Note**: It can take some time to generate embeddings for thousands of images.
For ease of use, by default, this colab uses precomputed embeddings. You can
also calculate them from scratch again by updating the relevant param in the
"Global params" section.

### This notebook is for API demonstration purposes only

**Note: This notebook is for API demonstration purposes only.**

It's important to use evaluation datasets
that reflect the expected distribution of images and patients you wish to use any downstream models on.

This means that the best way to determine if this API is right for you is to try it with data that would be used for the downstream task you're interested in.

**Note**: If you want to jump to training a model with embeddings, you can
scroll down to [Train a model with the embeddings from NLST](#train-nlst)

# Data Attribution

This notebook makes use of two public datasets provided by the Cancer Imaging Archive which is managed by the United States  National Cancer Institute

###  NLST Radiology CT Images CC BY 4.0
[https://www.cancerimagingarchive.net/collection/nlst/](https://www.cancerimagingarchive.net/collection/nlst/)

#### NLST Data Citation
 National Lung Screening Trial Research Team. (2013). Data from the National Lung Screening Trial (NLST) [Data set]. The Cancer Imaging Archive. https://doi.org/10.7937/TCIA.HMQ8-J677
### LIDC-IDRI Data Access CC BY 3.0
https://www.cancerimagingarchive.net/collection/lidc-idri/

#### LIDC-IDRI Data Citation

Armato III, S. G., McLennan, G., Bidaut, L., McNitt-Gray, M. F., Meyer, C. R., Reeves, A. P., Zhao, B., Aberle, D. R., Henschke, C. I., Hoffman, E. A., Kazerooni, E. A., MacMahon, H., Van Beek, E. J. R., Yankelevitz, D., Biancardi, A. M., Bland, P. H., Brown, M. S., Engelmann, R. M., Laderach, G. E., Max, D., Pais, R. C. , Qing, D. P. Y. , Roberts, R. Y., Smith, A. R., Starkey, A., Batra, P., Caligiuri, P., Farooqi, A., Gladish, G. W., Jude, C. M., Munden, R. F., Petkovska, I., Quint, L. E., Schwartz, L. H., Sundaram, B., Dodd, L. E., Fenimore, C., Gur, D., Petrick, N., Freymann, J., Kirby, J., Hughes, B., Casteele, A. V., Gupte, S., Sallam, M., Heath, M. D., Kuhn, M. H., Dharaiya, E., Burns, R., Fryd, D. S., Salganicoff, M., Anand, V., Shreter, U., Vastagh, S., Croft, B. Y., Clarke, L. P. (2015). Data From LIDC-IDRI [Data set]. The Cancer Imaging Archive. https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX

# Installation & Setup



In [ ]:
# Notebook specific dependencies
# !pip install google_health.ct_dicom
# TODO OPTIONAL: Create pip installation for the code below - https://github.com/Google-Health/google-health/tree/master/ct_dicom

!pip install absl-py dicomweb-client[gcp] google-auth requests-toolbelt
!pip install tf-models-official==2.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import io
import http
import matplotlib
import pydicom
import numpy as np
from PIL import Image
from typing import Iterable, Optional
from google.colab import auth
from google.oauth2 import credentials
import pandas as pd
import dicomweb_client.ext.gcp.uri as gcp_uri
import dicomweb_client.uri as dicomweb_uri
from google.colab import auth
from google.oauth2 import credentials
from google.auth import credentials as gcredentials
from google.auth.transport import requests
from requests_toolbelt.multipart import decoder
from google.cloud import storage
import tensorflow as tf
import matplotlib.pyplot as plt

**IMPORTANT**: If you are using Colab, you must restart the runtime after installing new packages.

NOTE: There will be some ERROR messages due to the protobuf library - this is normal.

In [ ]:
#@title Classes for testing data access and visualization

"""Google Cloud Healthcare (CHC) DICOMweb utilities."""

# Well-known constants from https://www.dicomstandard.org/.
_STUDY_INSTANCE_UID_TAG = '0020000D'
_SERIES_INSTANCE_UID_TAG = '0020000E'
_SOP_INSTANCE_UID_TAG = '00080018'

_SERIES_INSTANCE_UID_SEARCH_SUFFIX = 'series'
_STUDY_INSTANCE_UID_SEARCH_SUFFIX = 'studies'
_SOP_INSTANCE_UID_SEARCH_SUFFIX = 'instances'

_VALUE_KEY = 'Value'

# Scope requirements from:
# https://cloud.google.com/healthcare-api/docs/reference/rest/v1/projects.locations.datasets.dicomStores/searchForInstances#authorization-scopes
_AUTHORIZATION_SCOPES = ['https://www.googleapis.com/auth/cloud-healthcare']

# Search result limits for the CHC DICOMweb API:
# https://cloud.google.com/healthcare-api/docs/dicom#search_parameters
_MAX_LIMIT_STUDY = 5000
_MAX_LIMIT_SERIES = 5000
_MAX_LIMIT_SOP = 50000
_MAX_OFFSET = 1000000

_MAX_REFRESH_ATTEMPTS = 10
_REQUEST_TIMEOUT_SECONDS = 600


def create_authorized_session(
    credentials: gcredentials.Credentials,
) -> requests.AuthorizedSession:
  """Creates a Session authorized for Cloud Healthcare API interactions.

  Args:
    credentials: Google Auth credentials. For further details, see
      https://googleapis.dev/python/google-auth/latest/index.html.

  Returns:
    Credentials object with the requisite API scope.
  """
  authorization_scopes = _AUTHORIZATION_SCOPES
  scoped_credentials = gcredentials.with_scopes_if_required(
      credentials, authorization_scopes
  )
  return requests.AuthorizedSession(
      scoped_credentials, max_refresh_attempts=_MAX_REFRESH_ATTEMPTS
  )


def download_multipart_dicom_series(
    project_id: str,
    location: str,
    dataset_id: str,
    dicom_store_id: str,
    session: requests.AuthorizedSession,
    study_instance_uid: str,
    series_instance_uid: str,
) -> Iterable[bytes]:
  """Downloads all SOP Instances (DICOMs) within a Series Instance UID.

  The request accepts a multipart MIME response from the CHC DICOMweb API to
  reduce the:
  - Latency associated with making one API call per Instance.
  - API quota usage while downloading all Instances within a Series.

  Args:
    project_id: The GCP Project containing the DICOM Store to query.
    location: The regional location associated with the DICOM Store (c.f.
      https://cloud.google.com/healthcare-api/docs/regions).
    dataset_id: The Dataset containing the DICOM Store (c.f.
      https://cloud.google.com/healthcare-api/docs/projects-datasets-data-stores)
    dicom_store_id: The DICOM Store to query.
    session: An Google Auth session authorized to use the CHC DICOMweb API.
    study_instance_uid: The Study Instance UID containing the Series Instance
      UID to download.
    series_instance_uid: The Series Instance UID containing the SOP Instances
      (DICOMs) to download.

  Yields:
    DICOM bytes associated with each Instance contained within the input Series
    Instance UID.
  """
  dicomweb_path = str(
      dicomweb_uri.URI(
          str(
              gcp_uri.GoogleCloudHealthcareURL(
                  project_id, location, dataset_id, dicom_store_id
              )
          ),
          study_instance_uid,
          series_instance_uid,
      )
  )

  headers = {
      'Accept': (
          'multipart/related; transfer-syntax=1.2.840.10008.1.2.1;'
          ' type="application/dicom"'
      )
  }
  response = session.get(
      dicomweb_path, headers=headers, timeout=_REQUEST_TIMEOUT_SECONDS
  )
  response.raise_for_status()

  for part in decoder.MultipartDecoder.from_response(response).parts:
    yield part.content


def search_study_instance_uids(
    project_id: str,
    location: str,
    dataset_id: str,
    dicom_store_id: str,
    session: requests.AuthorizedSession,
    limit: int = 100,
) -> Iterable[str]:
  """Recovers all Study Instance UIDs from a CHC DICOM Store.

  Args:
    project_id: The GCP Project containing the DICOM Store to query.
    location: The regional location associated with the DICOM Store (c.f.
      https://cloud.google.com/healthcare-api/docs/regions).
    dataset_id: The Dataset containing the DICOM Store (c.f.
      https://cloud.google.com/healthcare-api/docs/projects-datasets-data-stores)
    dicom_store_id: The DICOM Store to query.
    session: An Google Auth session authorized to use the CHC DICOMweb API.
    limit: The number of Study Instance UIDs in the DICOM Store could be large.
      The UIDs are recovered in a paginated fashion, where each page of results
      (one page per query) includes at most `limit` values. The higher this
      value, the fewer the total number of requests, but each response would be
      larger. Depending on your network connection, set this value in the range
      1 through 5000 (both inclusive). This parameter impacts the speed and
      network bandwidth utilization, but not the values returned by the method.

  Yields:
    Study Instance UIDs from the DICOM Store.

  Raises:
    ValueError: If `limit` exceeds the max value of 5000 allowed by the CHC
      DICOMweb API (c.f.
      https://cloud.google.com/healthcare-api/docs/projects-datasets-data-stores)
  """
  if limit > _MAX_LIMIT_STUDY:
    raise ValueError(
        f'Request limit {limit} exceeds the CHC Search query request limit of'
        f' {_MAX_LIMIT_STUDY} for Study Instances.'
    )
  yield from _search_dicom_data(
      project_id,
      location,
      dataset_id,
      dicom_store_id,
      _STUDY_INSTANCE_UID_SEARCH_SUFFIX,
      _STUDY_INSTANCE_UID_TAG,
      session,
      limit,
  )


def search_series_instance_uids(
    project_id: str,
    location: str,
    dataset_id: str,
    dicom_store_id: str,
    session: requests.AuthorizedSession,
    study_instance_uid: Optional[str] = None,
    limit: int = 100,
) -> Iterable[str]:
  """Recovers all Series Instance UIDs from a CHC DICOM Store.

  The scope may be restricted to all Series within a fixed Study Instance
  UIDs (see `study_instance_uid` below).

  Args:
    project_id: The GCP Project containing the DICOM Store to query.
    location: The regional location associated with the DICOM Store (c.f.
      https://cloud.google.com/healthcare-api/docs/regions).
    dataset_id: The Dataset containing the DICOM Store (c.f.
      https://cloud.google.com/healthcare-api/docs/projects-datasets-data-stores)
    dicom_store_id: The DICOM Store to query.
    session: An Google Auth session authorized to use the CHC DICOMweb API.
    study_instance_uid: If provided, restricts the returned Series Instance UIDs
      to within this Study Instance UID.
    limit: The number of Study Instance UIDs in the DICOM Store could be large.
      The UIDs are recovered in a paginated fashion, where each page (query)
      includes at most `limit` values. The higher this value, the fewer the
      total number of requests, but each response would be larger. Depending on
      your network connection, set this value in the range 1 through 5000 (both
      inclusive).

  Yields:
    Series Instance UIDs from the DICOM Store (optionally within the scope of
    the input `study_instance_uid`, if provided).

  Raises:
    ValueError: If `limit` exceeds the max value of 5000 allowed by the CHC
      DICOMweb API (c.f.
      https://cloud.google.com/healthcare-api/docs/projects-datasets-data-stores)
  """
  if limit > _MAX_LIMIT_SERIES:
    raise ValueError(
        f'Request limit {limit} exceeds the CHC Search query request limit of'
        f' {_MAX_LIMIT_SERIES} for Series Instances.'
    )
  search_suffix = (
      _SERIES_INSTANCE_UID_SEARCH_SUFFIX
      if study_instance_uid is None
      else f'studies/{study_instance_uid}/series'
  )
  yield from _search_dicom_data(
      project_id,
      location,
      dataset_id,
      dicom_store_id,
      search_suffix,
      _SERIES_INSTANCE_UID_TAG,
      session,
      limit,
  )


def search_sop_instance_uids(
    project_id: str,
    location: str,
    dataset_id: str,
    dicom_store_id: str,
    session: requests.AuthorizedSession,
    study_instance_uid: Optional[str] = None,
    series_instance_uid: Optional[str] = None,
    limit: int = 1000,
) -> Iterable[str]:
  """Recovers all SOP Instance UIDs from a CHC DICOM Store.

  The scope may be restricted to all Series within a fixed:
  - Study Instance UID.
  - Study and Series Instance UID pair.

  (see `study_instance_uid` and `series_instance_uid` below).

  Args:
    project_id: The GCP Project containing the DICOM Store to query.
    location: The regional location associated with the DICOM Store (c.f.
      https://cloud.google.com/healthcare-api/docs/regions).
    dataset_id: The Dataset containing the DICOM Store (c.f.
      https://cloud.google.com/healthcare-api/docs/projects-datasets-data-stores)
    dicom_store_id: The DICOM Store to query.
    session: An Google Auth session authorized to use the CHC DICOMweb API.
    study_instance_uid: If provided, restricts the returned SOP Instance UIDs to
      within this Study Instance UID.
    series_instance_uid: If provided, restricts the returned Series Instance
      UIDs to within this Series Instance UID. The associated Study Instance UID
      must also be specified.
    limit: The number of SOP Instance UIDs in the DICOM Store could be large.
      The UIDs are recovered in a paginated fashion, where each page (query)
      includes at most `limit` values. The higher this value, the fewer the
      total number of requests, but each response would be larger. Depending on
      your network connection, set this value in the range 1 through 5000 (both
      inclusive).

  Yields:
    SOP Instance UIDs from the DICOM Store (optionally within the scope of
    the input `study_instance_uid` and/or `series_instance_uid`, if provided).

  Raises:
    ValueError: If
      - `limit` exceeds the max value of 50000 allowed by the CHC DICOMweb API
        (c.f.
        https://cloud.google.com/healthcare-api/docs/projects-datasets-data-stores).
      - `series_instance_uid` is specified by `study_instance_uid` is not.
  """
  if limit > _MAX_LIMIT_SOP:
    raise ValueError(
        f'Request limit {limit} exceeds the CHC Search query request limit of'
        f' {_MAX_LIMIT_SOP} for SOP Instances.'
    )

  if study_instance_uid is None and series_instance_uid is None:
    search_suffix = _SOP_INSTANCE_UID_SEARCH_SUFFIX
  elif series_instance_uid is None:
    search_suffix = f'studies/{study_instance_uid}/instances'
  elif study_instance_uid is None:
    raise ValueError(
        'Study Instance UID must be provided if Series Instance UID is'
        ' specified.'
    )
  else:
    search_suffix = (
        f'studies/{study_instance_uid}/series/{series_instance_uid}/instances'
    )
  yield from _search_dicom_data(
      project_id,
      location,
      dataset_id,
      dicom_store_id,
      search_suffix,
      _SOP_INSTANCE_UID_TAG,
      session,
      limit,
  )


def _search_dicom_data(
    project_id: str,
    location: str,
    dataset_id: str,
    dicom_store_id: str,
    query_suffix: str,
    dicom_tag: str,
    session: requests.AuthorizedSession,
    limit: int,
) -> Iterable[str]:
  """Generates DICOM UIDs from a CHC DICOM Store."""
  assert limit > 0

  uri = gcp_uri.GoogleCloudHealthcareURL(
      project_id, location, dataset_id, dicom_store_id
  )
  base_dicomweb_query_path = f'{uri}/{query_suffix}?includefield={dicom_tag}'
  headers = {'Content-Type': 'application/dicom+json; charset=utf-8'}

  # The CHC offset limit puts an upper bound on the Instance count, which is
  # also used to limit the number of iterations.
  for offset in range(0, _MAX_OFFSET, limit):
    dicomweb_query_path = (
        f'{base_dicomweb_query_path}&offset={offset}&limit={limit}'
    )

    response = session.get(
        dicomweb_query_path, headers=headers, timeout=_REQUEST_TIMEOUT_SECONDS
    )
    response.raise_for_status()
    # CHC DICOMweb API does not set a Warning response header on the last
    # available page:
    # https://cloud.google.com/healthcare-api/docs/dicom#search_parameters
    if response.status_code == http.HTTPStatus.NO_CONTENT:
      return

    for instance in response.json():
      assert dicom_tag in instance
      assert _VALUE_KEY in instance[dicom_tag]

      for value in instance[dicom_tag][_VALUE_KEY]:
        yield value

In [ ]:
# @title Authenticate
# Authenticate user for access. There will be a popup asking you to sign in with your user and approve access.
auth.authenticate_user()
TOKEN_ = !gcloud beta auth application-default print-access-token
TOKEN = TOKEN_[0]

# This is your token for accessing the API and CT Volumes.
# It's good for 1 hour until you need a new one.
TOKEN

In [ ]:
# @title Set DICOM store parameters
project_id='hai-cd3-foundations' # @param {type:"string"}
location='us-central1'  # @param {type:"string"}
dataset_id='ct3d'  # @param {type:"string"}
dicom_store_id='lidc-idri' # @param {type:"string"}

<a name="train-nlst"></a>
# Train a model with the embeddings from NLST

Here we have a full set of embeddings from the NLST dataset that you can download and train a cancer detection model.

## Collect the stored NPZ data from the cloud bucket

In [ ]:
# GCS bucket with data to read:
gcs_storage_client = storage.Client(project_id) # our bucket is in the same Google Cloud Project as the DicomStore
gcs_bucket_name = 'hai-cd3-foundations-ct3d-vault-entry'# @param {type:"string"}
gcs_bucket = gcs_storage_client.bucket(gcs_bucket_name)
tune_path = 'nlst/nlst_tune_with_labels.npz' # @param {type:"string"}
train_path = 'nlst/nlst_train_with_labels.npz' # @param {type:"string"}

def read_embeddings(path):
  blob = gcs_bucket.blob(path)
  with blob.open('rb') as f:
    data = np.load(f,allow_pickle=True)
    key = data.files[0]
    return pd.DataFrame.from_dict(data[key].item(), orient='index')

df_tune = read_embeddings(tune_path)
df_train = read_embeddings(train_path)

## Train and Evaluate a model using precomputed embeddings.

In [ ]:
# choose whether you want predict whether the screen leads to a positive lung cancer diagnosis within 1 or 2 years.
model_head = 'cancer_in_2' # @param ["cancer_in_1","cancer_in_2"]
# Get NumPy arrays from DataFrames
train_embeddings = df_train.embedding
train_labels = df_train[model_head].values
tune_embeddings = df_tune.embedding
tune_labels = df_tune[model_head].values

# Convert the NumPy arrays to ragged tensors
train_embeddings = tf.constant(list(train_embeddings))
tune_embeddings = tf.convert_to_tensor(list(tune_embeddings))
train_ds = tf.data.Dataset.from_tensor_slices((train_embeddings, train_labels))
eval_ds = tf.data.Dataset.from_tensor_slices((tune_embeddings, tune_labels))

In [ ]:
import tensorflow as tf
import tensorflow_models as tfm


def create_model(heads,
                 token_num=1,
                 embeddings_size=1408,
                 learning_rate=0.07,
                 end_lr_factor=1.0,
                 dropout=0.5,
                 loss_weights=None,
                 hidden_layer_sizes=[128, 32],
                 weight_decay=0.0001,
                 seed=None) -> tf.keras.Model:
  """
  Creates linear probe or multilayer perceptron using LARS.

  """
  inputs = tf.keras.Input(shape=(token_num * embeddings_size,))
  inputs_reshape = tf.keras.layers.Reshape((token_num, embeddings_size))(inputs)
  inputs_pooled = tf.keras.layers.GlobalAveragePooling1D(data_format='channels_last')(inputs_reshape)
  hidden = inputs_pooled
  # If no hidden_layer_sizes are provided, model will be a linear probe.
  for size in hidden_layer_sizes:
    hidden = tf.keras.layers.Dense(
        size,
        activation='relu',
        kernel_initializer=tf.keras.initializers.HeUniform(seed=seed),
        kernel_regularizer=tf.keras.regularizers.l2(l2=weight_decay),
        bias_regularizer=tf.keras.regularizers.l2(l2=weight_decay))(
            hidden)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    hidden = tf.keras.layers.Dropout(dropout, seed=seed)(hidden)
  output = tf.keras.layers.Dense(
      units=len(heads),
      activation='sigmoid',
      kernel_initializer=tf.keras.initializers.HeUniform(seed=seed))(
          hidden)

  outputs = {}
  for i, head in enumerate(heads):
    outputs[head] = tf.keras.layers.Lambda(
        lambda x: x[..., i:i + 1], name=head.lower())(
            output)

  model = tf.keras.Model(inputs, outputs)
  model.compile(
      optimizer=tfm.optimization.lars.LARS(
         learning_rate=.1),
      loss=dict([(head, 'binary_focal_crossentropy') for head in heads]),
      loss_weights=loss_weights or 1.0,
      weighted_metrics=[
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.TruePositives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.AUC(),
        tf.keras.metrics.AUC(curve='PR', name='auc_pr')])
  return model

In [ ]:
# Train the model
DIAGNOSIS = 'cancer_in_2'
model = create_model(
    [DIAGNOSIS]
)

model.fit(
    x=train_ds.batch(512).prefetch(tf.data.AUTOTUNE).cache(),
    validation_data=eval_ds.batch(32).cache(),
    epochs=35,
)

In [ ]:
import sklearn
import matplotlib.pyplot as plt


def plot_curve(x, y, auc, x_label=None, y_label=None, label=None):
  fig = plt.figure(figsize=(10, 10))
  plt.plot(x, y, label=f'{label} (AUC: %.3f)' % auc, color='black')
  plt.legend(loc='lower right', fontsize=18)
  plt.xlim([-0.01, 1.01])
  plt.ylim([-0.01, 1.01])
  if x_label:
    plt.xlabel(x_label, fontsize=24)
  if y_label:
    plt.ylabel(y_label, fontsize=24)
  plt.xticks(fontsize=12)
  plt.yticks(fontsize=12)
  plt.grid(visible=True)

In [ ]:
rows = []
for embeddings, label in eval_ds.batch(1):
  row = {
      f'{DIAGNOSIS}_prediction': model(embeddings)[DIAGNOSIS].numpy().flatten()[0],
      f'{DIAGNOSIS}_value': label.numpy().flatten()[0]
  }
  rows.append(row)
eval_df = pd.DataFrame(rows)

In [ ]:
%matplotlib inline
labels = eval_df[f'{DIAGNOSIS}_value'].values
predictions = eval_df[f'{DIAGNOSIS}_prediction'].values
false_positive_rate, true_positive_rate, thresholds = sklearn.metrics.roc_curve(
    labels,
    predictions,
    drop_intermediate=False)
auc = sklearn.metrics.roc_auc_score(labels, predictions)
plot_curve(false_positive_rate, true_positive_rate, auc, x_label='False Positive Rate', y_label='True Positive Rate', label=DIAGNOSIS)

# Test access to LIDC DICOM store and the API

Get a token that grants access to the DICOM store and use it to download a volume via the DICOMWEb API. Next, we can collect the embeddings from the
selected embedding.

**NOTE**: You can skip this section if you just want to train a model on NLST data using the embeddings.

## Download a CT volume to visualize

In [ ]:
# @title Create a Session via a token.
creds = credentials.Credentials(TOKEN)
session = create_authorized_session(creds)

In [ ]:
# @title List the study and series instance UIDs and download a single volume.

TOTAL_VOLUMES = 5  # Total number to list out.
VOLUME_TO_SHOW = 1 # Which ones to render below

study_uids = list(search_study_instance_uids(
    project_id=project_id,
    location=location,
    dataset_id=dataset_id,
    dicom_store_id=dicom_store_id,
    session=session))


corresponding_series_uids = []
for study_number, a_study_uid in enumerate(study_uids):
  a_series = list(search_series_instance_uids(project_id=project_id,
    location=location,
    dataset_id=dataset_id,
    dicom_store_id=dicom_store_id,
    session=session, study_instance_uid=a_study_uid))[0]
  corresponding_series_uids.append(a_series)
  if study_number == TOTAL_VOLUMES:
    break


volume_as_bytes = list(download_multipart_dicom_series(
      project_id='hai-cd3-foundations',
      location='us-central1',
      dataset_id='ct3d',
      dicom_store_id='lidc-idri',
      session=session,
      study_instance_uid=study_uids[VOLUME_TO_SHOW],
      series_instance_uid=corresponding_series_uids[VOLUME_TO_SHOW],
  ))

print(f'Total Slices in downloaded volume: {len(volume_as_bytes)}')


Total Slices in downloaded volume: 219


In [ ]:
#@title Render a single slice
SLICE_TO_RENDER = 30
example_dicom = pydicom.dcmread(io.BytesIO(volume_as_bytes[SLICE_TO_RENDER]))

arr_unsigned = example_dicom.pixel_array.copy()
arr_unsigned = arr_unsigned.astype(np.float32)
arr_unsigned[arr_unsigned <0] = 0
arr_unsigned[arr_unsigned >1000] = 1000
arr_unsigned = (arr_unsigned / 1000) * 255
arr_unsigned = arr_unsigned.astype(np.uint8)
Image.fromarray(arr_unsigned, mode='L')

## Call the API to compute embeddings for the selected volume.

In [ ]:
#@title Define method to call API and create an LIDC URL.

from google.api_core.client_options import ClientOptions
from google.api_core.retry import Retry
from google.cloud import aiplatform
from google.api_core import exceptions
_API_ENDPOINT = 'us-west1-aiplatform.googleapis.com'


def create_lidc_series_url(study_instance_uid, series_instance_uid):
  return ('https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/'
          'locations/us-central1/datasets/ct3d/dicomStores/lidc-idri/dicomWeb/'
          f'studies/{study_instance_uid}/series/{series_instance_uid}')


_RETRIABLE_TYPES = (
    exceptions.TooManyRequests,  # HTTP 429
    exceptions.InternalServerError,  # HTTP 500
    exceptions.BadGateway,  # HTTP 502
    exceptions.ServiceUnavailable,  # HTTP 503
    exceptions.DeadlineExceeded,  # HTTP 504
)


def _is_retryable(exc):
  return isinstance(exc, _RETRIABLE_TYPES)


retry_policy = Retry(predicate=_is_retryable)


def get_ct_embeddings(mytoken: str, dicom_urls: List[str]) -> List[Any]:
  """Calls the API to collect the embeddings from a given volume.

  Args:
    mytoken: The token to access the DICOM volume and API.
    dicom_urls: The list of Series-level DICOM URL to the CT volumes.

  Returns:
    The list of embeddings or errors generated by the service. Differences in
    Vertex end-point configurations may change the return type. The caller is
    responsible for interpreting this value and extracting the requisite
    data.

  """
  api_client = aiplatform.gapic.PredictionServiceClient(
      client_options=ClientOptions(api_endpoint=_API_ENDPOINT)
  )
  endpoint = api_client.endpoint_path(
      project='hai-cd3-foundations', location='us-central1', endpoint=300
  )

  # Create a single instance to send - you can send up to 5.
  instances = []
  for dicom_url in dicom_urls:
    instances.append({"dicom_path": dicom_url, "bearer_token": mytoken})

  response = api_client.predict(
      endpoint=endpoint, instances=[instance], retry=retry_policy, timeout=600
  )
  assert len(response.predictions) == len(dicom_urls)
  assert len(response.predictions[0]) == 3
  # You can get the model version for this prediction
  # response.predictions[0]['model_version']

  # Check for no errors
  responses = []
  for a_prediction in response.predictions:
    if a_prediction['error_response'] is not None:
      responses.append(a_prediction['error_response'])
    else:
      embeddings = np.array(a_prediction['embedding_result']['embedding'],
                            dtype=np.float32)
      assert embeddings.shape == (1408,), 'Unexpected embeddings shape.'
      responses.append(embeddings)

  return embeddings

In [ ]:
#@title Create a URL and call the API for the DICOM volume

TOKEN_ = !gcloud beta auth application-default print-access-token
TOKEN = TOKEN_[0]

my_url = create_lidc_series_url(study_uids[VOLUME_TO_SHOW],
                                corresponding_series_uids[VOLUME_TO_SHOW])
get_ct_embeddings(mytoken=TOKEN, dicom_url=my_url)
